<h6>We import the necessary packages</h6>

In [72]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

In [98]:
file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
dataframe = pd.read_csv(file_url)

In [99]:
dataframe.shape

(303, 14)

In [100]:
dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


<h6>Let's split the data into a training and validation set, where the validation set is 20% of the whole data.
then we only remove validation dataset rows by index from the dataframe to make a training set</h6>

In [101]:
val_dataframe = dataframe.sample(frac=0.2, random_state=1337)
train_dataframe = dataframe.drop(val_dataframe.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)

Using 242 samples for training and 61 for validation


<h6>Often when working on a deep learning model you will retrieve your source data in a pandas. DataFrame and need to convert it into a format that Tensorflow can read. Fortunately, Tensorflow now has Datasets which create data pipelines for your network to train on. Converting a DataFrame into a tf dataset</h6>

<h6>pop function returns item passed in parameter and drop it from the dataframe. </h6>

<h6>we can get the slices of an array in the form of objects by using tf.data.Dataset.from_tensor_slices() method.</h6>

<h6>The tf.data.Dataset.shuffle() method randomly shuffles a tensor along its first dimension, buffer_size is the number of elements from which the new dataset will be sampled.</h6>


In [117]:
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("target")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds


train_ds = dataframe_to_dataset(train_dataframe)
val_ds = dataframe_to_dataset(val_dataframe)

<h6>Each Dataset yields a tuple (input, target) where input is a dictionary of features and target is the value 0 or 1:</h6>

In [118]:
for x, y in train_ds.take(1):
    print("Input:", x)
    print("Target:", y)

Input: {'age': <tf.Tensor: shape=(), dtype=int64, numpy=62>, 'sex': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'cp': <tf.Tensor: shape=(), dtype=int64, numpy=4>, 'trestbps': <tf.Tensor: shape=(), dtype=int64, numpy=160>, 'chol': <tf.Tensor: shape=(), dtype=int64, numpy=164>, 'fbs': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'restecg': <tf.Tensor: shape=(), dtype=int64, numpy=2>, 'thalach': <tf.Tensor: shape=(), dtype=int64, numpy=145>, 'exang': <tf.Tensor: shape=(), dtype=int64, numpy=0>, 'oldpeak': <tf.Tensor: shape=(), dtype=float64, numpy=6.2>, 'slope': <tf.Tensor: shape=(), dtype=int64, numpy=3>, 'ca': <tf.Tensor: shape=(), dtype=int64, numpy=3>, 'thal': <tf.Tensor: shape=(), dtype=string, numpy=b'reversible'>}
Target: tf.Tensor(1, shape=(), dtype=int64)


<h6>The tf.data.Dataset.batch() function is used to group the elements into batches, where batchSize is the number elements that should there in a single batch. </h6>

In [119]:
train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)

In [120]:
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup

<h6>forcategorical features encoded as integers, we will encode them using One hot encoding, it is one method of converting data to prepare it for an algorithm and get a better prediction. With one-hot, we convert each categorical value into a new categorical column and assign a binary value of 1 or 0 to those columns. Each integer value is represented as a binary vector. All the values are zero, and the index is marked with a 1.
we want a simple solution that will handle out of range inputs at inference, so we will use IntegerLookup().
</h6>

<h6> We also have a categorical feature encoded as a string, We will create an index of all possible features and encode output using the StringLookup() layer
</h6>

<h6>Normalization can help training of our neural networks as the different features are on a similar scale, which helps to stabilize the gradient descent step, allowing us to use larger learning rates or help models converge faster for a given learning rate.</h6>

<h6>Calling adapt() on a Normalization layer is an alternative to passing in mean and variance arguments during layer construction. A Normalization layer should always either be adapted over a dataset or passed mean and variance .</h6>

In [54]:
def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = Normalization()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature

In [55]:
def encode_categorical_feature(feature, name, dataset, is_string):
    lookup_class = StringLookup if is_string else IntegerLookup
    # Create a lookup layer which will turn strings into integer indices
    lookup = lookup_class(output_mode="binary")

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the set of possible string values and assign them a fixed integer index
    lookup.adapt(feature_ds)

    # Turn the string input into integer indices
    encoded_feature = lookup(feature)
    return encoded_feature

<h6>In Keras, the input layer itself is not a layer, but a tensor. It's the starting tensor you send to the first hidden layer.</h6>

In [64]:
# Categorical features encoded as integers
sex = keras.Input(shape=(1,), name="sex", dtype="int64")
cp = keras.Input(shape=(1,), name="cp", dtype="int64")
fbs = keras.Input(shape=(1,), name="fbs", dtype="int64")
restecg = keras.Input(shape=(1,), name="restecg", dtype="int64")
exang = keras.Input(shape=(1,), name="exang", dtype="int64")
ca = keras.Input(shape=(1,), name="ca", dtype="int64")

# Categorical feature encoded as string
thal = keras.Input(shape=(1,), name="thal", dtype="string")

# Numerical features
age = keras.Input(shape=(1,), name="age")
trestbps = keras.Input(shape=(1,), name="trestbps")
chol = keras.Input(shape=(1,), name="chol")
thalach = keras.Input(shape=(1,), name="thalach")
oldpeak = keras.Input(shape=(1,), name="oldpeak")
slope = keras.Input(shape=(1,), name="slope")

<h6>layers.concatenate, is a function that concatenates a list of inputs. It takes as input a list of tensors, all of the same shape except for the concatenation axis, and returns a single tensor, the concatenation of all inputs.</h6>

<h6>A Dense layer feeds all outputs from the previous layer to all its neurons, each neuron providing one output to the next layer.</h6>

<h6>Dropout is a technique to reduce overfitting when training the model.The choice of neurons to be deactivated is random. We assign a probability p to all neurons which determines their activation.
When p = 0.2, each neuron has 2/10 to be deactivated.</h6>

<h6>keras.Model groups layers into an object, where inputs are a list of keras.Input objects and the outputs of the model</h6> 

In [121]:
all_inputs = [
    sex,
    cp,
    fbs,
    restecg,
    exang,
    ca,
    thal,
    age,
    trestbps,
    chol,
    thalach,
    oldpeak,
    slope,
]
# Integer categorical features
sex_encoded = encode_categorical_feature(sex, "sex", train_ds, False)
cp_encoded = encode_categorical_feature(cp, "cp", train_ds, False)
fbs_encoded = encode_categorical_feature(fbs, "fbs", train_ds, False)
restecg_encoded = encode_categorical_feature(restecg, "restecg", train_ds, False)
exang_encoded = encode_categorical_feature(exang, "exang", train_ds, False)
ca_encoded = encode_categorical_feature(ca, "ca", train_ds, False)

# String categorical features
thal_encoded = encode_categorical_feature(thal, "thal", train_ds, True)

# Numerical features
age_encoded = encode_numerical_feature(age, "age", train_ds)
trestbps_encoded = encode_numerical_feature(trestbps, "trestbps", train_ds)
chol_encoded = encode_numerical_feature(chol, "chol", train_ds)
thalach_encoded = encode_numerical_feature(thalach, "thalach", train_ds)
oldpeak_encoded = encode_numerical_feature(oldpeak, "oldpeak", train_ds)
slope_encoded = encode_numerical_feature(slope, "slope", train_ds)

all_features = layers.concatenate(
    [
        sex_encoded,
        cp_encoded,
        fbs_encoded,
        restecg_encoded,
        exang_encoded,
        slope_encoded,
        ca_encoded,
        thal_encoded,
        age_encoded,
        trestbps_encoded,
        chol_encoded,
        thalach_encoded,
        oldpeak_encoded,
    ]
)
x = layers.Dense(32, activation="relu")(all_features)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(all_inputs, output)
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])

In [66]:
# `rankdir='LR'` is to make the graph horizontal.
keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [67]:
model.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50
8/8 [==============================] - 1s 34ms/step - loss: 0.8434 - accuracy: 0.3967 - val_loss: 0.8244 - val_accuracy: 0.2459
Epoch 2/50
8/8 [==============================] - 0s 3ms/step - loss: 0.7972 - accuracy: 0.4463 - val_loss: 0.7597 - val_accuracy: 0.4262
Epoch 3/50
8/8 [==============================] - 0s 3ms/step - loss: 0.7394 - accuracy: 0.5041 - val_loss: 0.7049 - val_accuracy: 0.5902
Epoch 4/50
8/8 [==============================] - 0s 3ms/step - loss: 0.7095 - accuracy: 0.5124 - val_loss: 0.6559 - val_accuracy: 0.6721
Epoch 5/50
8/8 [==============================] - 0s 3ms/step - loss: 0.6231 - accuracy: 0.6446 - val_loss: 0.6155 - val_accuracy: 0.7049
Epoch 6/50
8/8 [==============================] - 0s 3ms/step - loss: 0.6191 - accuracy: 0.6736 - val_loss: 0.5809 - val_accuracy: 0.7541
Epoch 7/50
8/8 [==============================] - 0s 3ms/step - loss: 0.5664 - accuracy: 0.7273 - val_loss: 0.5510 - val_accuracy: 0.7705
Epoch 8/50
8/8 [=================